<a href="https://colab.research.google.com/github/fberetta8/ML_in_Finance_Spring_2025_ETH/blob/main/Solution9_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Indifference pricing in Heston model

In [ ]:
# Need a certain version of numpy

!pip uninstall numpy -y
!pip cache purge
!pip install numpy==1.26.0 --no-cache-dir

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
print(np.__version__)
import torch

torch.manual_seed(42)
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Default device:", DEVICE)

In [ ]:
# In each epoch, N_PATHS brownian motion time-series are generated.
N_PATHS = 50000
# How many times a model is updated in the experiment.
N_EPOCHS = 100

In [ ]:
!pip install pfhedge

In [ ]:
from pfhedge.instruments import EuropeanOption, HestonStock, BrownianStock
from pfhedge.nn import Hedger, MultiLayerPerceptron

In [ ]:
heston = HestonStock(
    cost=0.0,           # No transaction cost
    dt=1/252,           # Daily time steps
    kappa=1.0,
    theta=0.04,         # Long-term volatility
    sigma=0.2,          # Vol of vol
    rho=-0.7,           # Correlation between price and vol
)

heston.simulate(n_paths=N_PATHS, time_horizon=30/252)

In [ ]:
# Prepare a derivative to hedge
derivative = EuropeanOption(heston, strike=1.0, maturity=30/252, call=True)

# Create your hedger
model = MultiLayerPerceptron()
hedger = Hedger(
    model, inputs=["log_moneyness", "expiry_time", "volatility", "prev_hedge"]
)

# Fit and price
hedger.fit(derivative, n_epochs=N_EPOCHS, n_paths=N_PATHS)
indifference_price = hedger.price(derivative)


Loss=0.0276: 100%|██████████| 100/100 [04:55<00:00,  2.96s/it]

Indifference Price: 0.0271


In [ ]:
# Price by deep hedging
print(f"Indifference Price: {indifference_price:.4f}")

Indifference Price: 0.0271


In [ ]:
# Simple montecarlo runs
mc_price = torch.mean(derivative.payoff()).item()
print(f"Monte Carlo Indifference price: {mc_price:.4f}")

Monte Carlo Expected Payoff: 0.0258
